In [ ]:
# Python 
from datetime import datetime
import ipywidgets as widgets
import warnings
warnings.filterwarnings(action = 'ignore') 

# Thrid part
from datetime import datetime, timedelta
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.style.use('seaborn') # ggplot
plt.rcParams['figure.figsize'] = [15, 5]
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 20)
pd.options.display.float_format = '{:,.2f}'.format
import re
from scipy import stats
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
sns.set_theme(style="darkgrid")

# Time series
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Activation, Dropout, Bidirectional ##
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import pmdarima as pm
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, month_plot, quarter_plot
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt, ARMA

# Support modules
from support import Utils as UT
from support.Utils import add_value_labels
from support import DL

In [ ]:
# parameters

FILEPATH = './data/Backup_orders_after_preparation.csv'

TODAY = datetime.today()
DATE_START_HISTORY = (2017,1,1)
DATE_END_HISTORY = (TODAY.year,TODAY.month,TODAY.day)

TEST_SIZE = 3 # MONTHS

# For out of sample forecasts 
PERIODS = 6 # MONTHS

In [ ]:
# Load orders file
try:
    df = UT.load_orders(FILEPATH)
except Exception as e:
    print(e)

# Obtain the time series using the support function - RESAMPLED MONTHS
timeseries = UT.make_pivot_orders_channel(
                    df
                    , period = 'M'
                    , date_start = DATE_START_HISTORY
                    , date_end = DATE_END_HISTORY
                    , roll = 12
                    ).Total

timeseries.shape

In [ ]:
# Make train and test sets

y = timeseries.values
X = timeseries.index.values

X_train = X[:-TEST_SIZE]
X_test = X[-TEST_SIZE:]
y_train = y[:-TEST_SIZE]
y_test = y[-TEST_SIZE:]

In [ ]:
timeseries.describe()

In [ ]:
# Plot the data adding resample on a quarterly base and rolling view

WINDOW = 3

fig = plt.figure()
fig.set_size_inches(18,8)
plt.plot(timeseries, 'b')
plt.plot(timeseries.rolling(window=WINDOW).mean(), 'r')
plt.plot(timeseries.rolling(window=WINDOW).std(), 'y')
plt.title('Time series')
plt.legend(['Total by month', 'Rolling mean window = {}'.format(WINDOW), 'Rolling std dev window = {}'.format(WINDOW)])
plt.show()

In [ ]:
# FB Prophet

## REMOVING OUTLIERS 

ts = timeseries.copy()
ts[ts > ts.describe()['75%']] = ts.describe()['75%']
ts[ts < ts.describe()['25%']] = ts.describe()['25%']

# define cap and floor
CAP = ts.rolling(window=WINDOW).mean().max()
FLOOR = ts.rolling(window=WINDOW).mean().min()

# Re-organize the time series for Prophet
prophet_df = pd.DataFrame({'ds': ts.index.values, 'y': ts.values})

# define cap and floor
prophet_df['cap'] = CAP
prophet_df['floor'] = FLOOR

# create the object
m = Prophet(growth='logistic')

## add holidays
# m.add_country_holidays(country_name='US')
## m.train_holiday_names

# Fit the model
m.fit(prophet_df)

# Create the time series for prediction using Prophet make_future_dataframe
with_out_of_sample_dates_df = m.make_future_dataframe(periods=PERIODS, freq='M')

with_out_of_sample_dates_df['cap'] = CAP
with_out_of_sample_dates_df['floor'] = FLOOR

# Predict
forecast_prohpet = m.predict(with_out_of_sample_dates_df)

# Plot forecast and the components
fig1 = m.plot(forecast_prohpet)
fig1.set_size_inches(15,5)
ax = fig1.gca()
ax.set(ylabel='Monthly revenue', title='Forecast')

fig2 = m.plot_components(forecast_prohpet)
fig2.set_size_inches(15,5)

print('\n sqrt mse: {:,.7}'.format(np.sqrt(MSE(ts[-TEST_SIZE:], forecast_prohpet.set_index('ds').loc[ts.index[-TEST_SIZE:], 'yhat']))))

In [ ]:
# Interactive plot
plot_plotly(m, forecast_prohpet[:-1], figsize=(1100, 600))

In [ ]:
# Projections

forecast_to_year_end = forecast_prohpet.set_index('ds').loc['2021-8':'2021-12', 'yhat'].sum()
# actual from timeseries instead of ts in which I removed the outliers
actual_plus_forecast = timeseries['2021-01-01':].sum() + forecast_to_year_end

print(f"The Prophet mean forecast is to the end of the year is: {forecast_to_year_end:,.2f}")
print(f'The total volume of the business: {actual_plus_forecast:,.2f}')

# -------------------------------------------

In [ ]:
# Plot ACF and PACF to find ARMA coeffs for the estimated residuals

fig, axs = plt.subplots(3,1, figsize=(15,15))
plot_acf(X, title='acf', ax = axs[0])
plot_pacf(X, title='pacf', ax = axs[1])
month_plot(timeseries, ax = axs[2]) # timeseries
axs[2].set_title('Seasonal plot of monthly data')
plt.show()

In [ ]:
# SARIMA

SARIMA_PERIODICITY = 3 # MONTHS

# Plot autocorrelation and partial autocorrelation functions
def plots(data, lags=None):
    plt.figure()
    layout = (1, 3)
    raw  = plt.subplot2grid(layout, (0, 0))
    acf  = plt.subplot2grid(layout, (0, 1))
    pacf = plt.subplot2grid(layout, (0, 2))
    
    raw.plot(data)
    sm.tsa.graphics.plot_acf(data, lags=lags, ax=acf, zero=False)
    sm.tsa.graphics.plot_pacf(data, lags=lags, ax=pacf, zero = False)
    sns.despine()
    plt.tight_layout()

# Fit the model
sar = sm.tsa.statespace.SARIMAX(timeseries, 
                                order=(2,0,0), 
                                seasonal_order=(0,1,1,SARIMA_PERIODICITY), 
                                trend='c').fit()
# display(sar.summary())

# Plot summary statistics for the residual curve
plots(sar.resid[sar.loglikelihood_burn:], lags=20);

In [ ]:
# Plot diagnostocs which show the normality and no autocorrelation
sar.plot_diagnostics(lags=12,figsize = (15,10));

In [ ]:
# Predict SARIMA
preds_sarima = sar.predict(start = 1, end= len(X_test))

print('sqrt mse: {:,.7}'.format(np.sqrt(MSE(y_test, preds_sarima))))

In [ ]:
# Deep Learning RNN and LSTM

# Create the dataframe
df_timeseries = pd.DataFrame(timeseries)

# Scale the data
scaler = MinMaxScaler()
scaler.fit(df_timeseries)
df_scaled = pd.DataFrame(np.reshape(scaler.transform(df_timeseries), (-1,)), columns=['Total'], index=df_timeseries.index)

# Define parameters which the support functions use to create the train and test splits
SERIES_PERIODS = timeseries.shape[0]
INPUT_PERIODS = 6
TEST_PERIODS = 12
SAMPLE_GAP = 1

# Train and test split scaled for training and predicting
X_train_dl_sc, X_test_dl_init_sc, y_train_dl_sc, y_test_dl_sc = DL.get_train_test_data(df_scaled, 'Total'
                                                                , SERIES_PERIODS, INPUT_PERIODS, TEST_PERIODS, SAMPLE_GAP)

print('Training input shape: {}'.format(X_train_dl_sc.shape))
print('Training output shape: {}'.format(y_train_dl_sc.shape))
print('Test input shape: {}'.format(X_test_dl_init_sc.shape))
print('Test output shape: {}'.format(y_test_dl_sc.shape))

In [ ]:
# Fit a simple RNN
history, model1 = DL.fit_SimpleRNN(X_train_dl_sc, y_train_dl_sc, cell_units=70, epochs=1500)

display(model1.summary())

# Predict with RNN
preds_rnn_sc = DL.predict(X_test_dl_init_sc, n_steps=len(y_test_dl_sc), model=model1)

# Inverse scaling
y_test_dl = scaler.inverse_transform(np.reshape(y_test_dl_sc,(-1,1)))
preds_rnn = scaler.inverse_transform(preds_rnn_sc)

print('\n','sqrt mse: {:,.7}'.format(np.sqrt(MSE(y_test_dl, preds_rnn))))

In [ ]:
# Plot the prediction vs the train and test data
plt.figure()
DL.predict_and_plot(X_test_dl_init_sc, y_test_dl_sc, model1,
                 'Total sales: Test Data and Simple RNN Predictions', TEST_PERIODS)

In [ ]:
FIGSIZE = (16,8)

out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

tab = widgets.Tab(children = [out1, out2, out3])
tab.set_title(0, 'time series')
tab.set_title(1, 'train test')
tab.set_title(2, 'out-of-sample')

with out1:
    plt.figure(figsize=FIGSIZE)
    plt.plot(X, y, 'o-r', label='actual')
    plt.plot(forecast_prohpet.set_index('ds').loc[:, 'yhat'][:-PERIODS], 'x-b', label='Prophet')
    plt.plot(X, sar.predict(start = 1, end= len(y)), 'x-y', label='SARIMA')
    plt.plot(X, scaler.inverse_transform(DL.predict(X_test_dl_init_sc, n_steps=len(timeseries), model=model1)), 'x-g', label='RNN')
    plt.legend(loc = 'best')
    plt.title('Time series')
    plt.show()

with out2:
    plt.figure(figsize=FIGSIZE)
    plt.plot(X_train[-TEST_SIZE * 2:], y_train[-TEST_SIZE * 2:], 'o-b', label='train')
    plt.plot(X_test, y_test, 'o-r', label='test')
    plt.plot(X_test, forecast_prohpet.set_index('ds').loc[with_out_of_sample_dates_df.set_index('ds').index[-TEST_SIZE:]
                                                          , 'yhat'], 'x-k', label='Prophet')
    plt.plot(X_test, sar.predict(start = 1, end= len(y_test)), 'x-y', label='SARIMA')
    plt.plot(X_test, scaler.inverse_transform(DL.predict(X_test_dl_init_sc, n_steps=TEST_SIZE, model=model1)), 'x-g', label='RNN')
    plt.legend(loc = 'best')
    plt.title('Train and test sets')
    plt.show()

with out3:
    plt.figure(figsize=FIGSIZE)
    plt.plot(forecast_prohpet.set_index('ds').loc[with_out_of_sample_dates_df.set_index('ds').index[-PERIODS:]
                                 , 'yhat'], 'x-b', label='Prophet')
    plt.plot(with_out_of_sample_dates_df.set_index('ds').index[-PERIODS:]
                                 , sar.predict(start = 1, end = PERIODS), 'x-y', label='SARIMA')
    plt.plot(with_out_of_sample_dates_df.set_index('ds').index[-PERIODS:]
                                 , scaler.inverse_transform(DL.predict(X_test_dl_init_sc, n_steps=PERIODS, model=model1)), 'x-k', label='RNN')
    plt.legend(loc = 'best')
    plt.title('Out of sample forecasts')
    plt.show()
    
display(tab)

In [ ]:
prophet_forecast = forecast_prohpet.set_index('ds').loc[with_out_of_sample_dates_df.set_index('ds').index[-PERIODS:]
                                                          , 'yhat']

RNN_forecast = scaler.inverse_transform(DL.predict(X_test_dl_init_sc, n_steps=PERIODS, model=model1))

SARIMA_forecast = sar.predict(start = 1, end = PERIODS)

print('Prophet forecast: ${:,.0f}'.format(np.sum(prophet_forecast)))

print('SARIMA forecast: ${:,.0f}'.format(np.sum(SARIMA_forecast)))

print('RNN forecast: ${:,.0f}'.format(np.sum(RNN_forecast)))

print('Average forecast: ${:,.0f}'.format(np.mean((np.sum(RNN_forecast.flatten()), np.sum(prophet_forecast), np.sum(SARIMA_forecast)))))